### **aperiodic_irasa**

Separate aperiodic activity from 1-60 Hz power spectra using the IRASA method, and compute the aperiodic slope and offset. 

IRASA described in: Wen, H, Liu, Z. Separating fractal and oscillatory components in the power spectrum of neurophysiological signal. *Brain Topography* 29 (2016). [https://doi.org/10.1007/s10548-015-0448-0](https://doi.org/10.1007/s10548-015-0448-0) 

Author: EL Johnson, PhD

Copyright (c) 2022-2025  
ZR Cross, PhD & EL Johnson, PhD

### Download the sample data file:
[https://drive.google.com/file/d/1KBlYuFC8-K31LRf22ueaTliSrNZENwnZ/view?usp=sharing](https://drive.google.com/file/d/1KBlYuFC8-K31LRf22ueaTliSrNZENwnZ/view?usp=sharing)

### Set directories:

In [ ]:
datdir = 'C:\\Users\\elizl\\Downloads\\' # replace with your path to the downloaded data
savdir = datdir # replace with your save path

### Import modules:

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

!pip install mne
import mne

!pip install yasa
import yasa

### Read sample data file:

In [ ]:
fid = 'NM01_new.mat-epo.fif.gz'
fid_pth = os.path.join(datdir, fid)
epochs = mne.read_epochs(fid_pth, preload = True)

### Set subject ID and task:

In [ ]:
sid = fid.split('_')[0] # extract subject ID from sample file ID
task = 'WM' # WM/Scene/Rest - the sample dataset is WM
print(f'Ready to analyze subject {sid}.')

### Extract info from the epoched data:

In [ ]:
data = epochs.get_data(tmin = 0, tmax = 3) # 0 to 3 s from stimulus onset
sf = epochs.info['sfreq'] # sampling rate
chan = epochs.ch_names # channel names

### Run IRASA:

In [ ]:
# initialize data frames
dfs = []
psd_total = []
df_aperiodic = []

# loop through trials
for idx in range(data.shape[0]):
    freqs, psd_aperiodic, psd_osc, fit_params = yasa.irasa(data[idx, :, :], sf, 
                                                               ch_names = chan, 
                                                               band = (1, 60), 
                                                               win_sec = 1, 
                                                               return_fit = True)
    
    # generate data frame for fit parameters
    fit_params.insert(loc = 0, column = 'epoch', value = idx) # add epoch column
    fit_params['subj'] = sid # add subject column
    fit_params['task'] = task # add task column
    dfs.append(fit_params) # append each epoch data frame
    
    # append the PSD arrays
    psd_total.append(psd_osc)
    
    # generate data frame for aperiodic component
    df_aperiodic_epoch = pd.DataFrame(psd_aperiodic)
    df_aperiodic_epoch.insert(loc = 0, column = 'epoch', value = idx)
    df_aperiodic_epoch['subj'] = sid # add subject column
    df_aperiodic_epoch['task'] = task # add task column
    df_aperiodic.append(df_aperiodic_epoch) # append each epoch data frame

### Save fit parameters:

In [ ]:
df = pd.concat(dfs)
df.to_csv(f'{savdir}fit_params_trial_{sid}_{task}.csv', sep = ',', mode = 'a', header = True)
df.head() # show top of the saved table

### Save aperiodic components:

In [ ]:
df_aperiodic_df = pd.concat(df_aperiodic)
df_aperiodic_df.to_csv(f'{savdir}aperiodic_psd_trial_{sid}_{task}.csv', sep = ',', mode = 'a', header = True)
df_aperiodic_df.head() # show top of the saved table

### Average fit parameters across trials and save:

In [ ]:
df.set_index('Chan', inplace = True)
cols_to_average = ['Intercept', 'Slope', 'R^2', 'std(osc)']
df_mean = df.groupby(df.index)[cols_to_average].mean()
df_mean['subj'] = sid
df_mean['task'] = task
df_mean.to_csv(f'{savdir}fit_params_mean_{sid}_{task}.csv', sep = ',', mode = 'a', header = True)
df_mean.head() # show top of the saved table

### Average aperiodic components across trials and save:

In [ ]:
df_aperiodic_df.set_index(df.index, inplace = True)
cols_to_average = df_aperiodic_df.columns[1:61]
df_aperiodic_mean = df_aperiodic_df.groupby(df_aperiodic_df.index)[cols_to_average].mean()
df_aperiodic_mean['subj'] = sid
df_aperiodic_mean['task'] = task
df_aperiodic_mean.to_csv(f'{savdir}aperiodic_psd_mean_{sid}_{task}.csv', sep = ',', mode = 'a', header = True)
df_aperiodic_mean.head() # show top of the saved table

### Plot the aperiodic component of the first channel:

In [ ]:
dat = df_aperiodic_mean.iloc[0,:60].tolist()
plt.plot(freqs, dat, 'k', lw = 2.5)
plt.xlim(1, 60)
plt.yscale('log') # log-log scale
plt.title(f'aperiodic component at {df_aperiodic_mean.index[0]}', fontsize = 15)
plt.xlabel('frequency [Hz]', fontsize = 15)
plt.ylabel('log(PSD)', fontsize = 15)